In [ ]:
pip install fastparquet

In [ ]:
import git

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import image
import matplotlib.dates as mdates
from scipy import stats
from datetime import timedelta

In [ ]:
import torch

In [ ]:
import sys
print(sys.executable)

In [ ]:
PATH_TO_EPDE = "C:\\Users\\Ksenia\\jupyter310\\Lib\\site-packages\\epde\\"
sys.path.append(PATH_TO_EPDE)

In [ ]:
import epde

In [ ]:
from epde.control import ControlExp, ConstrLocation, ConditionalLoss, ControlConstrEq, ControlConstrNEq
from epde.interface.prepared_tokens import DerivSignFunction

In [ ]:
from epde.interface.interface import EpdeMultisample

In [ ]:
epde.__dict__

In [ ]:
file_path = Path(r'C:\Users\Ksenia\NSS\ODE_projects\air_qual')  

data_file = file_path / 'AllYearsES1.parquet'
data = pd.read_parquet(data_file)

In [ ]:
station = 'Cidade Continental' #change

data = data[data['Estacao'] == station]
data['datetime'] = pd.to_datetime(data['Data'] + ' ' + data['Hora'], format='mixed', errors='coerce')
data = data.dropna(subset=['datetime'])
data.set_index('datetime', inplace=True)
data = data.sort_index()

In [ ]:
mp_10 = data[data['Poluente'] == 'MP10']
df = mp_10[['Valor']].dropna()

In [ ]:
hours_to_epde = 72

In [ ]:
time_end = df.index.max()
time_start = time_end - timedelta(hours=hours_to_epde)
time_data = df[time_start:time_end].copy()
full_index = pd.date_range(
    start=time_data.index.min().floor('T'),
    end=time_data.index.max().ceil('T'),
    freq='5T'  
)
regular_data = pd.DataFrame(index=full_index)
time_data = time_data.combine_first(regular_data)
time_data = time_data[~time_data.index.duplicated(keep='first')]
time_data = time_data.sort_index()
value = time_data['Valor'].interpolate(method='time')


In [ ]:
# EPDE initialization for one exact station
bnd = 1000
n_epochs = 100
popsize = 5
max_axis_idx = x.ndim - 1
t = np.arange(0, len(x))  # Time values

diff_mode = 'FD'

# Initialize EPDE search object
epde_search_obj = epde.EpdeSearch(use_solver=False, multiobjective_mode=True,
                                  boundary=bnd, dimensionality=max_axis_idx,
                                  coordinate_tensors=[t, ])

# Set equation factors limits
factors_max_number = {'factors_num': [1, 2], 'probas': [0.6, 0.4]}

# Set differentiation mode
if diff_mode == 'ANN':
    epde_search_obj.set_preprocessor(default_preprocessor_type='ANN',
                                     preprocessor_kwargs={'epochs_max': 50000})
elif diff_mode == 'poly':
    epde_search_obj.set_preprocessor(default_preprocessor_type='poly',
                                     preprocessor_kwargs={'use_smoothing': False, 
                                                          'sigma': 1,
                                                          'polynomial_window': 3, 
                                                          'poly_order': 3})
elif diff_mode == 'FD':
    epde_search_obj.set_preprocessor(default_preprocessor_type='FD')
else:
    raise NotImplementedError('Incorrect differentiation mode selected.')

# Define tokens for EPDE
trig_tokens = epde.TrigonometricTokens(freq=(0.95, 1.05), dimensionality=max_axis_idx)

# Set MOEA/DD parameters for EPDE
epde_search_obj.set_moeadd_params(population_size=popsize, training_epochs=n_epochs)

# Perform EPDE fitting
epde_search_obj.fit(data=[value], variable_names=['u'], max_deriv_order=(2,),
                    equation_terms_max_number=4, data_fun_pow=2,
                    additional_tokens=[trig_tokens],
                    equation_factors_max_number=factors_max_number,
                    eq_sparsity_interval=(1e-12, 1e-10))

# Extract and display the resulting equations
res = epde_search_obj.equations(True)
print(res)

# Save `res` to a CSV file for further use
# res.to_csv('results.csv')

In [ ]:
num_samples = 817
monte_carlo_samples = pd.DataFrame(index=value.index, columns=value.columns)
for column in df.columns:
    mean_value = df[column].mean()
    std_dev = df[column].std()
    min_value = df[column].min()
    max_value = df[column].max()
    for i in range(num_samples):
        current_mean = df[column].iloc[i % df.shape[0]]
        current_std_dev = std_dev * 0.01
        noise = np.random.normal(loc=0, scale=current_std_dev)
        sample = current_mean + noise
        sample = np.clip(sample, min_value, max_value)
        monte_carlo_samples.loc[i, column] = sample

monte_carlo_samples = monte_carlo_samples.dropna()

# Calculate pairwise distances and select similar rows
distances = pairwise_distances(df, monte_carlo_samples)
threshold = 2.0
similar_rows = [monte_carlo_samples.iloc[j] for j in range(distances.shape[1]) if distances[0][j] < threshold]
new_coeff = pd.DataFrame(similar_rows)
new_coeff.columns = monte_carlo_samples.columns


def construct_general_equation_dict(new_coeff):
    equations = []
    for _, row in new_coeff.iterrows():
        # Eequations based on terms observed in the dataset
        equation_dict = {
            'u{power: 1.0}': {'coeff': row.get('u{power: 1.0}', 0.0), 
                               'term': [[None]], 
                               'pow': [1.0], 
                               'var': [0]},
            'du/dx0{power: 1.0}': {'coeff': row.get('du/dx0{power: 1.0}', 0.0), 
                                    'term': [[0]], 
                                    'pow': [1.0], 
                                    'var': [0]},
            'd^2u/dx0^2{power: 1.0}': {'coeff': row.get('d^2u/dx0^2{power: 1.0}', 0.0), 
                                        'term': [[1]], 
                                        'pow': [1.0], 
                                        'var': [0]},
            'u{power: 2.0}': {'coeff': row.get('u{power: 2.0}', 0.0), 
                              'term': [[None]], 
                              'pow': [2.0], 
                              'var': [0]},
            ' cos{power: 1.0}': {'coeff': row.get( 'cos{power: 1.0}', 0.0), 
                                                  'term': [[2]], 
                                                  'pow': [2.0, 1.0], 
                                                  'var': [0]},

        }
        
        # Append the dictionary for this equation
        equations.append(equation_dict)
    
    return equations

eqs = [Equation() for i in range(10)]
    for eq_idx, eq in enumerate(eqs):
        eq.add(equations[eq_idx])

def build_ann() -> torch.nn.Sequential:
    """Creates a feedforward neural network with 3 hidden layers using Tanh activation."""
    return torch.nn.Sequential(
        torch.nn.Linear(2, 100),  # Input layer (2 features) -> first hidden layer (100 neurons)
        torch.nn.Tanh(),         # Activation (Tanh)
        torch.nn.Linear(100, 100),  # First hidden layer -> second hidden layer
        torch.nn.Tanh(),         # Activation (Tanh)
        torch.nn.Linear(100, 100),  # Second hidden layer -> third hidden layer
        torch.nn.Tanh(),         # Activation (Tanh)
        torch.nn.Linear(100, 1)  # Third hidden layer -> output layer (1 neuron)
    )

   #Build one ANN per equation
   anns = [build_ann() for _ in eqs]
    c_cache = cache.Cache(cache_verbose=False, model_randomize_parameter=1e-6)
    cb_es = early_stopping.EarlyStopping(eps=1e-5,
                                         loss_window=100,
                                         no_improvement_patience=1000,
                                         patience=5,
                                         randomize_parameter=1e-10,
                                         info_string_every=500
                                         )
    cb_plots = plot.Plots(save_every=None, print_every=None)
    # Optimizer for model training
    optimizer = Optimizer('Adam', {'lr': 1e-3})

print(f'eqs are {eqs}')
start = time.time()
for eq_idx, equation in enumerate(eqs):
    model = Model(anns[eq_idx], domain, equation, boundaries)  # batch_size = 390
    # print('batch size', model.batch_size)
    model.compile('NN', lambda_operator=1, lambda_bound=100)
    model.train(optimizer, 3000, save_model=False, callbacks=[cb_es, c_cache, cb_plots])
    end = time.time()
    print('Time taken 10= ', end - start)

    solutions = []
    for net_idx, net in enumerate(anns):
        anns[net_idx] = net.to(device=device_type())
        solutions.append(anns[net_idx](domain.build('NN')))  # .detach().numpy().reshape(-1))
        solutions_tensor = torch.stack(solutions, dim=0)  # Tensor containing all solutions
print(f"Solutions tensor shape: {solutions_tensor.shape}")
average_solution_tensor = solutions_tensor.mean(dim=0)
average_solution = average_solution_tensor.detach().numpy().reshape(-1)  # Reshape to 1D for saving
#Save solutions to results storage
pt_directory = r''
os.makedirs(pt_directory, exist_ok=True)
solution_file_name = f"several_solutions_{len(solutions)}_shape_{solutions_tensor.shape}.pt"
torch.save(solutions_tensor, os.path.join(pt_directory, pt_file_name))